In [65]:
# NOT COMPLETE AND NOT ACCURATE 
# topgyal gurung
# KNN with breast cancer dataset 

import numpy as np 
import math
import pandas as pd
import warnings
from collections import Counter
import random

train_file='/Users/topgyalgurung/Desktop/capstone-project/KNN/breast-cancer-dataset/train.csv'
test_file='/Users/topgyalgurung/Desktop/capstone-project/KNN/breast-cancer-dataset/test.csv'
validate_file='/Users/topgyalgurung/Desktop/capstone-project/KNN/breast-cancer-dataset/val.csv'


In [42]:
train_data=pd.read_csv(train_file)
test_data=pd.read_csv(test_file, header=None)
validate_data=pd.read_csv(validate_file, header=None)

train_data.drop(['id'],1, inplace=True)
test_data.drop([0],1, inplace=True)
validate_data.drop([0],1, inplace=True)


In [43]:
train_data.head(2)

,clump_thick,uniform_cell_size,uniform_cell_shape,margin_adhesion,sing_epithelial_cellsize,nuclei,chromatin,nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2


In [44]:
test_data.head(2)

,1,2,3,4,5,6,7,8,9,10
0,2,1,1,1,2,1,1,1,1,2
1,3,1,1,1,2,1,2,1,1,2


In [45]:
validate_data.head(2)

,1,2,3,4,5,6,7,8,9,10
0,4,1,1,1,2,1,3,1,1,2
1,6,1,3,2,2,1,1,1,1,2


In [46]:
train_data=train_data.astype(float).values.tolist()
test_data=test_data.astype(float).values.tolist()
validate_data=validate_data.astype(float).values.tolist()

In [47]:
# dictionaries have to keys: 2 for benign tumors and 4 for malignant
train_set={2:[],4:[]}  
test_set={2:[],4:[]}
validate_set={2:[],4:[]}

for i in train_data:
    train_set[i[-1]].append(i[:-1]) # appending list to last element
for i in test_data:
    test_set[i[-1]].append(i[:-1])
for i in validate_data:
    validate_set[i[-1]].append(i[:-1])

In [48]:
from pprint import pprint
#pprint(train_set)

In [64]:
pprint(train_set[2])

[[5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
 [5.0, 4.0, 4.0, 5.0, 7.0, 10.0, 3.0, 2.0, 1.0],
 [3.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 1.0, 1.0],
 [6.0, 8.0, 8.0, 1.0, 3.0, 4.0, 3.0, 7.0, 1.0],
 [4.0, 1.0, 1.0, 3.0, 2.0, 1.0, 3.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 2.0, 10.0, 3.0, 1.0, 1.0],
 [2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
 [2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 5.0],
 [4.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0],
 [2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 1.0, 1.0],
 [4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
 [4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
 [6.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
 [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
 [3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0],
 [5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
 [2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
 [1.0, 1.0, 3.0, 1.0, 2.0, 1.0, 1.0, 1

In [50]:
pprint(train_set[4])

[[8.0, 10.0, 10.0, 8.0, 7.0, 10.0, 9.0, 7.0, 1.0],
 [5.0, 3.0, 3.0, 3.0, 2.0, 3.0, 4.0, 4.0, 1.0],
 [8.0, 7.0, 5.0, 10.0, 7.0, 9.0, 5.0, 5.0, 4.0],
 [7.0, 4.0, 6.0, 4.0, 6.0, 1.0, 4.0, 3.0, 1.0],
 [10.0, 7.0, 7.0, 6.0, 4.0, 10.0, 4.0, 1.0, 2.0],
 [7.0, 3.0, 2.0, 10.0, 5.0, 10.0, 5.0, 4.0, 4.0],
 [10.0, 5.0, 5.0, 3.0, 6.0, 7.0, 7.0, 10.0, 1.0],
 [5.0, 2.0, 3.0, 4.0, 2.0, 7.0, 3.0, 6.0, 1.0],
 [10.0, 7.0, 7.0, 3.0, 8.0, 5.0, 7.0, 4.0, 3.0],
 [10.0, 10.0, 10.0, 8.0, 6.0, 1.0, 8.0, 9.0, 1.0],
 [5.0, 4.0, 4.0, 9.0, 2.0, 10.0, 5.0, 6.0, 1.0],
 [2.0, 5.0, 3.0, 3.0, 6.0, 7.0, 7.0, 5.0, 1.0],
 [10.0, 4.0, 3.0, 1.0, 3.0, 3.0, 6.0, 5.0, 2.0],
 [6.0, 10.0, 10.0, 2.0, 8.0, 10.0, 7.0, 3.0, 3.0],
 [5.0, 6.0, 5.0, 6.0, 10.0, 1.0, 3.0, 1.0, 1.0],
 [10.0, 10.0, 10.0, 4.0, 8.0, 1.0, 8.0, 10.0, 1.0],
 [3.0, 7.0, 7.0, 4.0, 4.0, 9.0, 4.0, 8.0, 1.0],
 [7.0, 8.0, 7.0, 2.0, 4.0, 8.0, 3.0, 8.0, 2.0],
 [9.0, 5.0, 8.0, 1.0, 2.0, 3.0, 2.0, 1.0, 5.0],
 [5.0, 3.0, 3.0, 4.0, 2.0, 4.0, 3.0, 4.0, 1.0],
 [10.0, 3.0, 6.0

In [62]:
pprint(test_set)

{2: [[2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [1.0, 1.0, 1.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 2.0, 3.0, 4.0, 1.0, 1.0, 1.0],
     [1.0, 2.0, 1.0, 3.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0],
     [4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [5.0, 4.0, 5.0, 1.0, 8.0, 1.0, 3.0, 6.0, 1.0],
     [1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
     [1.0, 1.0, 3.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [1.0, 1.0, 3.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 3.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [1.0, 1

In [52]:
pprint(validate_set)

{2: [[4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
     [6.0, 1.0, 3.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0],
     [4.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [1.0, 1.0, 3.0, 2.0, 2.0, 1.0, 3.0, 1.0, 1.0],
     [5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
     [5.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0],
     [4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [6.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [5.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0],
     [3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [5.0, 3.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0],
     [4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [2.0, 1.0, 3.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0],
     [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
     [3.0, 1

In [53]:
#data to train, new data to predict and a value for k
def k_nearest_neighbors(data,predict,k):
    distances=[]  # list of distances
    for group in data: # each class in data
        for features in data[group]: # iterating through features
            euclidean_distance = math.sqrt( (features[0]-predict[0])**2 + (features[1]-predict[1])**2 )
            distances.append([euclidean_distance,group])  
    votes=[i[1] for i in sorted(distances)[:k]]  
    vote_result=Counter(votes).most_common(1)[0][0] 
    return vote_result

In [54]:
len(train_data)
train_size=len(train_data)

In [55]:
# test with different odd k values since binary classification
k_range=range(1,train_size)
correct=0
total=0
max_accuracy=0.0
optimal_k=0
accuracy=0.0
accuracies=[]
for k in k_range:
    # iterate through groups in testset
    for group in validate_set:
        #go through data point , test on validate_set
        for data in validate_set[group]:
            # feed data point into function 
            vote=k_nearest_neighbors(train_set, data, k)
            # if group from test_set equal to vote from knn classifier
            if group==vote:
                correct+=1
            total+=1 
            accuracy=correct/total      
    print('Accuracy with ',k, accuracy)
    if max_accuracy<accuracy:
            max_accuracy=accuracy
            optimal_k=k   
    accuracies.append(accuracy)

Accuracy with  1 0.9555555555555556
Accuracy with  2 0.9555555555555556
Accuracy with  3 0.9604938271604938
Accuracy with  4 0.9666666666666667
Accuracy with  5 0.9718518518518519
Accuracy with  6 0.9740740740740741
Accuracy with  7 0.9767195767195768
Accuracy with  8 0.9787037037037037
Accuracy with  9 0.980246913580247
Accuracy with  10 0.9814814814814815
Accuracy with  11 0.9824915824915825
Accuracy with  12 0.9833333333333333
Accuracy with  13 0.9834757834757835
Accuracy with  14 0.9841269841269841
Accuracy with  15 0.9846913580246913
Accuracy with  16 0.9851851851851852
Accuracy with  17 0.9856209150326798
Accuracy with  18 0.9860082304526749
Accuracy with  19 0.98635477582846
Accuracy with  20 0.9866666666666667
Accuracy with  21 0.9869488536155203
Accuracy with  22 0.9872053872053872
Accuracy with  23 0.9874396135265701
Accuracy with  24 0.9876543209876543
Accuracy with  25 0.9878518518518519
Accuracy with  26 0.9880341880341881
Accuracy with  27 0.9882030178326474
Accuracy with

In [59]:
# median of k ?? 
print('max accuracy: ', max_accuracy)
print('optimal k: ',optimal_k)

max accuracy:  0.9907770515613653
optimal k:  102


In [60]:
# test with test_set with optimal k and find accuracy
for group in test_set:
        #go through data point , test on validate_set
    for data in test_set[group]:
            # feed data point into function 
        vote=k_nearest_neighbors(train_set, data, optimal_k)
        if group==vote:
            correct+=1
        total+=1
        accuracy=correct/total
print('Accuracy: ', accuracy)

Accuracy:  0.8715313942823387
